# Homework 1

We start by importing the packages we will need. Notice that we fix the seed of the random number generator so everyone obtains the same results.

In [ ]:
using JuMP
using Cbc
using Random
Random.seed!(2)         # Control random number generation
using LinearAlgebra
using Plots
using Statistics
using Test

We generate the instances at random, given a predefined number of potential suppliers ($i \in I$), demand points ($j \in J$) and time periods ($t \in T$), all parameters are randomly generated.

In [ ]:
# This structure will ease the passing of the specific instance for the functions 
# that generate and solve the optimization model.
mutable struct Instance
    nI     # Number of suppliers      
    nJ     # Number of demand points
    nT     # Number of periods
    I      # Supplier range
    J      # Demand points range
    T      # Periods range
    C      # Unit capacity costs per supplier
    H      # Unit storage cost per supplier
    M      # Production cost per supplier
    D      # Client demands in all periods
    Q      # Unit costs of unfulfilled demand
    F      # Unit costs to fulfil demands 
end

## Task 1.1

The function ```solve_deterministic()``` generates and solves the deterministic model. It is good practice to wrap procedures that might be potentially called multiple times in functions. We also consider a keyword argument (following the semicolon) to turn off any printing in the code. However, there is [a bug](https://github.com/jump-dev/Cbc.jl/issues/168) with Cbc causing this parameter to not work.

You are required to complete this function by 
1. Adding the model variables
2. Adding the model objective
3. Adding the model constraints.

Notice that the object ```model``` has been already predefined.

In [ ]:
function solve_deterministic(ins::Instance; verbose = true)
    
    ## Renaming for making the implementation clearer
    I = ins.I
    J = ins.J
    T = ins.T 
    C = ins.C
    H = ins.H
    M = ins.M
    D = ins.D
    Q = ins.Q
    F = ins.F
    
    
    model = Model(Cbc.Optimizer)                        # We use Cbc solver

    ## Variables
# TODO: add your code here





    
    ## Objective: Minimize the total costs
# TODO: add your code here






    
    ## Constraints
# TODO: add your code here











    
    if !verbose
        set_silent(model)                     # Omit solver log
    end
    optimize!(model)                          # Solve the problem
    status = termination_status(model)        # Solution status
    if verbose
        println("Model status = $(status)")   # Print status
    end
    
    return (x,p,k,e,u)
end;

In [ ]:
## Test data
nI = 5                                 # Number of suppliers      
nJ = 5                                 # Number of demand points
nT = 5                                 # Number of periods
I = 1:nI                               # Supplier range
J = 1:nJ                               # Demand points range
T = 1:nT                               # Periods range

## Generate random data for the problem
C = ones(nI).*5                        # Unit capacity costs per supplier
H = ones(nI).*0.1                      # Unit storage cost per supplier
M = ones(nI)                           # Production cost per supplier

D = zeros(nJ,nT)
for j in J
    for t in T
        D[j,t] = j+0.05*(t-1)          # Client demands in all periods
    end
end

Q = ones(nJ).*50                       # Unit costs of unfulfilled demand

F = zeros(nI,nJ)
for i in I
    for j in J
        F[i,j] = abs(i-j)              # Unit costs to fulfill demands
    end
end

# This packages the problem instance information into a single structure.
test_ins = Instance(nI, nJ, nT, I, J, T, C, H, M, D, Q, F);

In [ ]:
(x_det_test,p_det_test,k_det_test,e_det_test,u_det_test) = solve_deterministic(test_ins, verbose = false)

@test all(value.(x_det_test).data .≈ [1.1; 2.1; 3.1; 4.1; 5.1])

## Data generation

We will now generate the data according to the description in task 1.2. We have predefined common growth rate $\mu$ and maximum deviation $\sigma$ to be used in the demand scenario process. The demand is then created as a first-order auto-regressive process. We wrap the process of creating the demand scenarios in the function ```create_scenarios(nS)``` that take as an argument the number of scenarios used ```nS```.

In [ ]:
## Generating the demand scenarios
function create_scenarios(ins::Instance, nS)

    ## Renaming for making the implementation clearer
    nJ = ins.nJ
    nT = ins.nT
    J = ins.J
    T = ins.T 
    D = ins.D
    
    S  = 1:nS                 # scenario set
    Ps = repeat([1/nS],nS )   # scenario probability

    ## d_sto: Stochastic demand
    D_sto = zeros(nS, size(D)[1], size(D)[2])    

    ## Creating the Monte Carlo simulation
    α = D[:,1]                           # Initial demand for each node
    μ = 0.05                             # Expected demand growth
    σ = 0.05                             # Max variability
    ϵ = randn(nS,nJ,nT)                  # This is the variability, following a standard normal 

    ## Assigning stochastic values
    for s in S
        for j in J
            D_sto[s,j,1] = (1 + σ * ϵ[s,j,1]) * α[j]
            for t in T[T.>1]
                D_sto[s,j,t] =  (1 + μ + σ * ϵ[s,j,t]) * D_sto[s,j,t-1]
            end
        end
    end
    return D_sto, Ps
end;

## Task 1.2

You are required to complete the function ```solve_stochastic()``` below. 


Just like in Task 1.1, you are required to complete this function by 
1. Adding the model variables
2. Adding the model objective
3. Adding the model constraints.

Notice that the object ```model``` has been already predefined and that the total number of scenarios ```nS``` as well as the probabilities ```Ps``` and demands ```d_sto``` for all scenarios are given as an arguments of the function.

In [ ]:
function solve_stochastic(ins::Instance, nS, Ps, D_sto; verbose=true)

    ## Renaming for making the implementation clearer    
    I = ins.I
    J = ins.J
    T = ins.T 
    C = ins.C
    H = ins.H
    M = ins.M
    Q = ins.Q
    F = ins.F
    
    S = 1:nS    # set of scenarios 
    
    model = Model(Cbc.Optimizer)      

    ## Variables
# TODO: add your code here






    ## Objective: Minimize the total first-stage + second-stage costs over all scenarios
# TODO: add your code here








    ## Constraints
# TODO: add your code here











    
    if verbose 
        println("Solving stochastic model with $(nS) scenarios...")    
    else
        set_silent(model)                     # Omit solver log
    end
    optimize!(model)                           # Solve the problem

    status = termination_status(model)         # Solution status
    
    if verbose 
        println(status)                        # Print status
    end
    
    return (x,p,k,e,u)
end;

In [ ]:
# DO NOT create the scenarios (run this cell) more than once or the test below won't work because the randomly generated data is different.
D_sto_test, Ps_test = create_scenarios(test_ins, 10);

In [ ]:
(x_sto_test, p_sto_test, k_sto_test, e_sto_test, u_sto_test) = solve_stochastic(test_ins, 10, Ps_test, D_sto_test, verbose = false)

@test all(round.(value.(x_sto_test).data, digits=2) .≈ [1.11; 2.11; 3.46; 4.65; 6.17]) # Use this for Julia 1.6 and earlier
# @test all(round.(value.(x_sto_test).data, digits=2) .≈ [1.12; 2.21; 3.53; 4.51; 5.71]) # Use this for Julia 1.7

## Homework instance

This is the data you are asked to use in you comparisons in Homework 1.2c). The dimensions are considerably larger than in the test instances above.

In [ ]:
## Problem data
nI = 25                                # Number of suppliers      
nJ = 25                                # Number of demand points
nT = 10                                # Number of periods
I = 1:nI                               # Supplier range
J = 1:nJ                               # Demand points range
T = 1:nT                               # Periods range

## Generate random data for the problem
C = rand(20:200, nI)                   # Unit capacity costs per supplier
H = rand(1:4, nI)                      # Unit storage cost per supplier
M = rand(10:40, nI)                    # Production cost per supplier
D = repeat(rand(100:500, nJ),1,nT)     # Client demands in all periods
D = (D'.*collect(range(1, step=0.05, length=nT)))' # Adding a trend to demand
Q = rand(5000:10000, nJ)               # Unit costs of unfulfilled demand
F = rand(3:45, (nI,nJ))                # Unit costs to fulfill demands

# This packages the problem instance information into a single structure.
ins = Instance(nI, nJ, nT, I, J, T, C, H, M, D, Q, F);

### Deterministic model

In [ ]:
## Solve the deterministic model
(x_det,p_det,k_det,e_det,u_det) = solve_deterministic(ins, verbose = false);

In [ ]:
xsol_det = value.(x_det)                          # Get optimal x values (reserved capacities)
xsol_det = round.(xsol_det.data, digits = 2)      # Round to 2 decimals 
fval_det = dot(C, xsol_det)                       # Optimal cost of reserved capacities

## Print optimal solution
println("Optimal solution (non-displayed x values are zero):\n")
for i = 1:length(xsol_det)
    if xsol_det[i] > 0.0
        println("x[$(i)] = $(xsol_det[i])")
    end
end

## Print optimal cost of reserved capacities
println("\nOptimal cost of reserved capacities: ", fval_det)

### Stochastic model
The cell below allows for plotting all the demand profiles (each demand scenario) for a predefined location $i$. You can observe how the demand behave through time and how the variability increases as we look further in the future.

In [ ]:
## Considering 50 scenarios for this study
nS = 50
D_sto, Ps = create_scenarios(ins, nS);

## Plotting the scenarios for a single locatiom
location = 1  # selected location for plotting

## Creating empty plot
plt = plot(
    xlabel = "Time periods",
    ylabel = "Demand",  
    title = "Demand - Location $(location)",
    legend = false, fmt = :png
)

## Including each demand series in the plot
for s in 1:nS                                 
    plot!(D_sto[s,location,:], legend = false)
end

## Plotting the expected demand 
plot!(sum(Ps[s]*D_sto[s,location,:] for s in 1:nS), 
    lw = 2,          # line weight
    ls = :dash,      # line stroke
    color = :black, 
    legend = false
)

plt

In [ ]:
## Solve the stochastic model
(x_sto, p_sto, k_sto, e_sto, u_sto) = solve_stochastic(ins, nS, Ps, D_sto, verbose = false);

In [ ]:
xsol_sto = value.(x_sto)                          # Get optimal x values (reserved capacities)
xsol_sto = round.(xsol_sto.data, digits = 2)      # Round to 2 decimals 
fval_sto = dot(C, xsol_sto)                       # Optimal cost of reserved capacities

## Print optimal solution
println("Optimal solution:\n")
for i = 1:length(xsol_sto)
    if xsol_sto[i] > 0.0
        println("x[$(i)] = $(xsol_sto[i])")
    end
end

## Print optimal cost of reserved capacities
println("\nOptimal cost of reserved capacities: ", fval_sto)

Notice how the solutions compare in terms of total capacity invested and number of nodes selected as suppliers. Are the capacity centres more centralised or more disperse? Can you think of a reason why (the latter is not relevant for the task, but an interesting thought exercise)? 

We also compare the total time (should be approximately the same as the model solution time) and the allocated memory using the Julia built-in ```@timed``` macro. That is one of the main reasons why we wrapped our model generation and solving into a function.

How do the two models compare in terms of computational requirements?

In [ ]:
times_det = []
allocs_det = []
n_sample = 5
for i in 1:n_sample
    stats_det = @timed solve_deterministic(ins, verbose = false);
    push!(times_det, stats_det.time)
    push!(allocs_det, stats_det.bytes)
end
mean_time_det = mean(times_det)
mean_allocs_det = mean(allocs_det)
times_det, allocs_det

In [ ]:
times_sto = []
allocs_sto = []
for i in 1:5
    stats_sto = @timed solve_stochastic(ins, nS, Ps, D_sto, verbose = false);
    push!(times_sto, stats_sto.time)
    push!(allocs_sto, stats_sto.bytes)
end
mean_time_sto = mean(times_sto)
mean_allocs_sto = mean(allocs_sto)
times_sto, allocs_sto

In [ ]:
println("Deterministic model solved in $(mean_time_det) seconds and allocated a total of $(mean_allocs_det/1E6)MB of memory.")
println("Stochastic model solved in $(mean_time_sto) seconds and allocated a total of $(mean_allocs_sto/1E6)MB of memory.")